# Import Packages

In [ ]:
!sudo apt-get update
!sudo apt-get install python3.8


[sudo] password for jupyter: 

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1 -f https://download.pytorch.org/whl/cu102/torch_stable.html

In [ ]:
!pip install tensorboard --user

In [ ]:
!pip install pycocotools --user

In [ ]:
!pip install addict

In [ ]:
import os
import cv2
import imageio
import torch

# Supplemental Functions

In [ ]:
# create a GIF from the images
def create_gif(image_dir, gif_path, duration = 1):
    images = []

    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        images.append(imageio.v2.imread(file_path))

    imageio.mimsave(gif_path, images, duration = duration)

In [ ]:
# create a mp4 video from the images
def create_video(image_dir, video_path, fps=1):
    images = []

    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        images.append(cv2.imread(file_path))

    height, width, layers = images[0].shape
    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for image in images:
        video.write(image)

    video.release()

# Clone Git Repo & Install Dependencies

In [ ]:
os.chdir("../Text Detection Project")

In [ ]:
!git clone https://github.com/meituan/YOLOv6.git
%cd YOLOv6
%pip install -r requirements.txt

# YOLOv6-S

In [ ]:
# download the pretrained model
torch.hub.download_url_to_file('https://github.com/meituan/YOLOv6/releases/download/0.4.0/yolov6s.pt', 'yolov6s.pt')

In [ ]:
%%time
# train custom data (single GPU) on train set
# caution: need to modify data/dataset.yaml to only one class ['text']
!python tools/train.py --batch 32 --conf configs/yolov6s_finetune.py --data data/dataset.yaml --fuse_ab --device 0

In [ ]:
%%time
# evaluation on validation set
# caution: need to modify best_ckpt.pt file path
!python tools/eval.py --data data/dataset.yaml  --weights runs/train/exp1/weights/best_ckpt.pt --task val --device 0

In [ ]:
%%time
# inference on test set
# caution: need to modify best_ckpt.pt file path
for i in range(1,101):
    filename = '../custom_dataset/images/test/test ' + str(i) + '.jpg'
    !python tools/infer.py --weights runs/train/exp1/weights/best_ckpt.pt --source '{filename}' --yaml data/dataset.yaml --device 0

In [ ]:
# make the images annotated by YOLO to a video for better checking
image_dir = 'runs/inference/exp'
video_path = 'runs/inference/exp/annotation video.mp4'
create_video(image_dir, video_path)

In [ ]:
# infer single images
# caution: need to modify best_ckpt.pt file path
!python tools/infer.py --weights runs/train/exp1/weights/best_ckpt.pt --source '../custom_dataset/images/test/infer 1.jpg' --yaml data/dataset.yaml --device 0
!python tools/infer.py --weights runs/train/exp1/weights/best_ckpt.pt --source '../custom_dataset/images/test/infer 2.jpg' --yaml data/dataset.yaml --device 0
# show image
img1 = cv2.imread('runs/inference/exp/infer 1.jpg')
cv2_imshow(img1)
img2 = cv2.imread('runs/inference/exp/infer 2.jpg')
cv2_imshow(img2)